# pianificazione di investimenti ver 2

Si vogliono realizzare $n$ progetti nei prossimi $T$ anni. Di ogni progetto i si conosce un indice di redditività $p_i$ che esprime il guadagno finale atteso (in Euro) e un profilo di costo $a_i$ = ($a_{i1}$, $a_{i2}$, ..., $a_{iT}$) per ogni anno del periodo considerato. 

Inoltre, in ogni anno $j$ del periodo si dispone di un budget di $b_j$ €. 

Quali progetti occorre selezionare per massimizzare il guadagno atteso rispettando i vincoli di budget?

![tab1](./tab1.png)

## tipo 1

Ogni nuovo progetto i comporta un costo globale di gestione $c_i$. 

Si vuole massimizzare il ricavo, cioè la differenza tra il guadagno atteso e i costi di gestione.

$$
Z = \max \left( \sum_T \left( \sum_i \left( P_i \ Pr_{T} \right) - \sum_i \left( a_{iT} \ Pr_{T} \right) \right) \right)
$$
$$

C1: \sum_i  \left( a_{iT} Pr_T \right) \le b_T \ \forall T \\

$$


In [3]:
from pyomo.environ import *
from pyomo.opt import SolverStatus, TerminationCondition

model = ConcreteModel()
model.dual = Suffix(direction=Suffix.IMPORT)

P = [30.0, 20.0, 25.0]
b = [30.0, 6.0, 6.0, 6.0]

c = [[10, 5, -2, -1], 
    [12, 2, -2, -5], 
    [15, -1, 5, 5]]

model.i = RangeSet(0,len(P)-1)
model.T = RangeSet(0,len(b)-1)

costi = {}
for i in model.i:
    for T in model.T:
        costi[i, T] = c[i][T]

P = {i: P[i] for i in model.i}
b = {T: b[T] for T in model.T}

model.P = Param(model.i, initialize=P)
model.b = Param(model.T, initialize=b)

model.c = Param(model.i, model.T, initialize=costi)

model.Pr = Var(model.i, model.T, domain=NonNegativeIntegers, initialize=0)

obj_expr = sum((sum(model.P[i]*model.Pr[i, T] for i in model.i) - sum(model.c[i, T]*model.Pr[i, T] for i in model.i)) for T in model.T)

model.ricavo = Objective(expr = obj_expr, sense=maximize)

model.constraints = ConstraintList()

for T in model.T:
    model.constraints.add(expr = sum(model.c[i, T]*model.Pr[i, T] for i in model.i) <= model.b[T])

#model.display()

results = SolverFactory('ipopt').solve(model)

results.write()

model.display()


    model.name="unknown";
      - termination condition: other
      - message from solver: <undefined>
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: warning
  Termination condition: other
Model unknown

  Variables:
    Pr : Size=12, Index=Pr_index
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        (0, 0) :     0 :     0 :  None : False : False : NonNegativeIntegers
        (0, 1) :     0 :     0 :  None : False : False : NonNegativeIntegers
        (0, 2) :     0 :     0 :  None : False : False : NonNegativeIntegers
        (0, 3) :     0 :     0 :  None : False : False : NonNegativeIntegers
        (1, 0) :     0 :     0 :  None : False : False : NonNegativeIntegers
